In [1]:
# Basic packages
import os
import torch
import numpy as np
import platform
# Import options
from config import get_options
from configx_cfg import get_cfg_options
# Import agent class
from designx import DesignX, ConfigX
# Import Modular-EC modules
from components.operators import *
from env.optimizer_env import Optimizer

from utils.utils import set_seed
from utils.make_dataset import get_test_problems
from utils.plots import display_optimizer
set_seed(2025)

Firstly let's load the models of Agent-1 and Agent-2:

In [2]:
# Load Agent-1
load_path_1 = "models/Agent-1.pt"
agent_1 = DesignX()
agent_1.load(load_path_1)
agent_1.eval()

# Load Agent-2
load_path_2 = "models/Agent-2.pt"
agent_2 = ConfigX()
agent_2.load(load_path_2)
agent_2.eval()

Load 116 sub-modules.
 [*] Loading data from models/Agent-1.pt
 [*] Loading data from models/Agent-2.pt


Now we get the 20 testing problems presented in the paper:

In [3]:
test_problems = get_test_problems()

Dataset generating: 100%|██████████| 12800/12800 [00:09<00:00, 1350.38it/s]


Then we calculate the ELA featuers of these problems:

In [4]:
ela_feature = agent_1.get_ELA_features(test_problems)

2025-05-16 12:34:42,353	INFO worker.py:1888 -- Started a local Ray instance.
/share/home/202210188530/DesignX_github/designx.py:177: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /opt/conda/conda-bld/pytorch_1716905987344/work/torch/csrc/utils/tensor_new.cpp:274.)
  ela_features = torch.concat([torch.tensor(elas).to(opts.device), torch.tensor(problem_info).float().to(opts.device)], -1).to(opts.device)


And construct the optimizer using Agent-1 with the ELA features

In [5]:
_, _, modules, _, _ = agent_1.forward(ela_feature, rollout=True)

Let's display the structure of the generated optimizer for the first problem:

In [6]:
display_optimizer(modules[0])

Halton_Init
 |
 |
Distance_Nich
 |-------------------------------------------------
 |                                                |                                                                                        
Pop_Size_50                                      Pop_Size_50                                                                               
 |                                                |                                               
 |                                                |                                                                                        
Multi_strategy                                   Multi_strategy                                                                            
 |   |-------------------------                   |   |-------------------------                  
 |   |        |               |                   |   |        |               |                                                           
 |  rand2    curre

We can see that the optimizer contains two DE sub-population with the same population sizes of 50 and the same modules. They use a multi-strategy mutation module consist with three DE mutations: rand2, current-to-best and current-to-rand, followed by the Binomial crossover. Then the offsprings are clipped at the bounds. After evaluation (omitted here) they are compared with their own parents and the better ones survive (DE_like selection). Both sub-popuations are then linearly reduced. Their sizes will be 5 at the end of optimization. 

Next we process the modules and construct the RL environment (Optimizer)

In [7]:
envs = []
for problem, module in zip(test_problems, modules):
    envs.append(Optimizer(problem, copy.deepcopy(module)))

Now we can optimize the problems with the generated optimizers controlled by Agent-2:

In [8]:
_, cost, _ = agent_2.rollout(envs)
print(np.mean(cost, -1))

[2.76765134e-01 9.95316205e-01 6.30237516e-07 7.13348330e+02
 8.11819934e-02 1.13021784e-01 4.79314537e-01 1.67693685e-01
 6.97923514e-01 1.84801786e+02 1.41202553e+00 1.09955729e+00
 1.24060114e+01 1.36108945e+01 2.20516120e-02 3.37304171e-02
 6.82325656e-04 1.72687261e-01 6.66170850e+00 8.95662427e+01]
